## Machine Learning Pipeline - Stroke Detection
- Name: Naufal Rahfi Anugerah
- Dicoding ID: nrahfi

## Import Required Library

In [1]:
# TensorFlow Library
import tensorflow as tf
import tensorflow_transform as tft

import pandas as pd
import zipfile as zf
import os

# TFX components for creating a TFX pipeline
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# TFX strategies and resolvers for working with model artifacts
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.dsl.components.common.resolver import Resolver 

# TFX standard artifacts for model and model blessing
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.types import Channel 

# TFX components for tuning and evaluating the model
from tfx.components import Tuner
from tfx.components import Evaluator

# TFX protocol buffer for training configurations
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2


## Data Configuration

### Download Dataset

In [2]:
# Download dataset using kaggle
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset
License(s): copyright-authors




  0%|          | 0.00/67.4k [00:00<?, ?B/s]
100%|██████████| 67.4k/67.4k [00:00<00:00, 205kB/s]
100%|██████████| 67.4k/67.4k [00:00<00:00, 205kB/s]


### Move Dataset

In [3]:
# Create raw directory then move the zip file into the folder
!mkdir raw
!move stroke-prediction-dataset.zip raw/

        1 file(s) moved.


### Extract Dataset

In [4]:
# Specify the path to the zip file containing the dataset
files = "raw/stroke-prediction-dataset.zip"

# Open the zip file in read mode
zip = zf.ZipFile(files, 'r')

# Extract all contents of the zip file to the 'raw/' directory
zip.extractall('raw/')

# Close the zip file after extraction
zip.close()

### Data Loading

In [5]:
# Load the data using pandas
df = pd.read_csv("raw/healthcare-dataset-stroke-data.csv")

# Show dataframe with full length
df.head(len(df))

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


### Data Assesing

In [6]:
# Show data type and memory usage for the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [7]:
def data_assessing(data):
    # Print the total number of NaN/Null values for each column, sorted in descending order
    print(f"Total NaN/Null Data per Column:\n{data.isna().sum().sort_values(ascending=False)}\n")
    
    # Print the shape of the data (number of rows and columns)
    print(f"Data Shape:\n{data.shape}")
    
    # Print the total number of duplicated rows in the data
    print(f"\nTotal Duplicated Data: {data.duplicated().sum()}")

# Call the function for checking the data is clean or not
data_assessing(df)

Total NaN/Null Data per Column:
bmi                  201
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
smoking_status         0
stroke                 0
dtype: int64

Data Shape:
(5110, 12)

Total Duplicated Data: 0


### Data Cleaning

In [8]:
# Fill the NaN column in every row for 'bmi' column with the mean value
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

In [9]:
# Call the function for checking the data is clean or not
data_assessing(df)

Total NaN/Null Data per Column:
id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

Data Shape:
(5110, 12)

Total Duplicated Data: 0


### Export Data

In [10]:
# Create data root directory
!mkdir data

# Export into csv format and store it in 'data' directory
df.to_csv("data/data.csv", index=False)

## Set Variables

In [11]:
# Initialize pipeline name and pipeline schema
PIPELINE_NAME = "stroke-pipeline"
SCHEMA_PIPELINE_NAME = "stroke-schema"

#Directory for saving artifacts
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [12]:
# Initialize root directory for the data
DATA_ROOT = "data"

In [13]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

In [14]:
# Define the output configuration for the CsvExampleGen component
# This configuration specifies how the input data should be split into training and evaluation sets
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            # Define the training split with 8 hash buckets
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            
            # Define the evaluation split with 2 hash buckets
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
        ]
    )
)

# Initialize the CsvExampleGen component with the input data directory and output configuration
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [15]:
# Initialize an interactive context for running TFX components interactively
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

### Create Statistic Summary

In [16]:
# Print out data examples
print(example_gen.outputs["examples"].get())

[Artifact(artifact: id: 1
type_id: 14
uri: "pipelines\\stroke-pipeline\\CsvExampleGen\\examples\\1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:336180,xor_checksum:1722617461,sum_checksum:1722617461"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.11.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DA

In [17]:
# Initialize the StatisticsGen component
# This component generates statistics for the dataset, which helps in understanding the data distribution
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
# Run the StatisticsGen component interactively
# This will compute and visualize the statistics in the interactive context
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [18]:
# Display the statistics generated by the StatisticsGen component
# This will provide a detailed view of the dataset statistics in the interactive context
interactive_context.show(statistics_gen.outputs["statistics"])

### Create Data Schema

In [19]:
# Initialize the SchemaGen component
# This component generates a schema based on the dataset statistics, which defines the expected data types and constraints for each feature
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)

# Run the SchemaGen component interactively
# This will infer the schema from the dataset statistics and make it available for further steps in the interactive context
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [20]:
# Display the schema generated by the SchemaGen component
# This will provide a detailed view of the inferred schema for the dataset in the interactive context
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Residence_type',STRING,required,,'Residence_type'
'age',FLOAT,required,,-
'avg_glucose_level',FLOAT,required,,-
'bmi',FLOAT,required,,-
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,-
'hypertension',INT,required,,-
'id',INT,required,,-


,Values
Domain,
'Residence_type',"'Rural', 'Urban'"
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


### Create Example Validator

In [21]:
# Initialize the ExampleValidator component
# This component validates the dataset against the generated schema and statistics to identify data quality issues
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

# Run the ExampleValidator component interactively
# This will validate the dataset according to the schema and statistics and provide insights into any data quality issues in the interactive context
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

### Checking Anomalies

In [22]:
# Display the anomalies detected by the ExampleValidator component
# This will provide a detailed view of any data quality issues or schema violations found during validation in the interactive context
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

In [23]:
# Initialize for creating transform module
TRANSFORM_MODULE_FILE = "transform.py"

In [24]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

# Dictionary defining categorical features and their number of categories
CATEGORICAL_FEATURES = {
    'gender': 2,
    'ever_married': 2,
    'work_type': 5,
    'Residence_type': 2,
    'smoking_status': 4
}

# List of numerical features
NUMERICAL_FEATURES = [
    'age',
    'hypertension',
    'heart_disease',
    'avg_glucose_level',
    'bmi'
]

# Key for the label feature
LABEL_KEY = 'stroke'

def transformed_name(key):
    """
    Generate a transformed feature name by appending '_xf' to the original key.

    Args:
        key (str): The original feature key.

    Returns:
        str: Transformed feature key.
    """
    return f"{key}_xf"

def convert_num_to_one_hot(label_tensor, num_labels=2):
    """
    Convert a label tensor (0 or 1) into a one-hot encoded tensor.

    Args:
        label_tensor (tf.Tensor): Tensor containing the label (0 or 1).
        num_labels (int, optional): Number of labels for one-hot encoding. Defaults to 2.

    Returns:
        tf.Tensor: One-hot encoded tensor with shape [batch_size, num_labels].
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features using TensorFlow Transform.

    Args:
        inputs (dict): Dictionary mapping feature keys to raw feature tensors.

    Returns:
        dict: Dictionary mapping feature keys to transformed feature tensors.
    """
    outputs = {}

    # Process categorical features
    for key, dim in CATEGORICAL_FEATURES.items():
        # Compute and apply vocabulary for categorical features
        int_value = tft.compute_and_apply_vocabulary(
            inputs[key], top_k=dim + 1
        )
        # Convert integer values to one-hot encoding
        outputs[transformed_name(key)] = convert_num_to_one_hot(
            int_value, num_labels=dim + 1
        )

    # Process numerical features
    for feature in NUMERICAL_FEATURES:
        # Scale numerical features to the range [0, 1]
        outputs[transformed_name(feature)] = tft.scale_to_0_1(inputs[feature])

    # Process label feature
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Overwriting transform.py


In [25]:
# Initialize the Transform component
# This component applies data transformations to the dataset according to the specified schema and transformation logic
transform = Transform(
    examples=example_gen.outputs["examples"],  # The examples output from the CsvExampleGen component
    schema=schema_gen.outputs["schema"],       # The schema output from the SchemaGen component
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)  # The path to the module file containing the transformation logic
)

# Run the Transform component interactively
# This will apply the specified transformations to the dataset and make the transformed data available in the interactive context
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\stroke-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\1afdc9aa121c41fc8ed686b1ddef1475\assets


INFO:tensorflow:Assets written to: pipelines\stroke-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\1afdc9aa121c41fc8ed686b1ddef1475\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Development

### Hyperparameter Tuner

In [26]:
# Initialize for creating tuner module
TUNER_MODULE_FILE = "tuner.py"

In [27]:
%%writefile {TUNER_MODULE_FILE}

from typing import NamedTuple, Dict, Text, Any

import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from keras import layers
from keras_tuner.engine import base_tuner
from transform import (
    CATEGORICAL_FEATURES,
    LABEL_KEY,
    NUMERICAL_FEATURES,
    transformed_name,
)

# Define a NamedTuple to hold the results of the tuning function
TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

# Early stopping callback to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    mode='max',
    verbose=1,
    patience=10
)

def gzip_reader_fn(filenames):
    """Read TFRecord files with GZIP compression.

    Args:
        filenames (str): Path to the TFRecord file(s).

    Returns:
        tf.data.TFRecordDataset: Dataset with GZIP-compressed data.
    """
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Create a dataset for training or evaluation.

    Args:
        file_pattern (str): Pattern for input TFRecord files.
        tf_transform_output (tft.TFTransformOutput): Transform output object.
        batch_size (int, optional): Number of samples per batch. Defaults to 64.

    Returns:
        tf.data.Dataset: Dataset containing (features, labels) tuples.
    """
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset

def get_tuner_model(hyperparameters, show_summary=True):
    """Define a Keras model for hyperparameter tuning.

    Args:
        hyperparameters (kt.HyperParameters): Hyperparameters for tuning.
        show_summary (bool, optional): Whether to print model summary. Defaults to True.

    Returns:
        tf.keras.Model: Compiled Keras model.
    """
    num_layers = hyperparameters.Choice('num_layers', values=[1, 2, 3])
    dense_units = hyperparameters.Int('dense_units', min_value=16, max_value=256, step=16)
    dropout_rate = hyperparameters.Float('dropout_rate', min_value=0.1, max_value=0.7, step=0.1)
    learning_rate = hyperparameters.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            layers.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            layers.Input(shape=(1,), name=transformed_name(feature))
        )

    concatenate = layers.concatenate(input_features)
    deep = layers.Dense(dense_units, activation='relu')(concatenate)

    for _ in range(num_layers):
        deep = layers.Dense(dense_units, activation='relu')(deep)

    deep = layers.Dropout(dropout_rate)(deep)
    outputs = layers.Dense(1, activation='sigmoid')(deep)

    model = tf.keras.Model(inputs=input_features, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['binary_accuracy']
    )

    if show_summary:
        model.summary()

    return model

def tuner_fn(fn_args):
    """Tune the model to find the best hyperparameters.

    Args:
        fn_args (NamedTuple): Contains information for tuning.

    Returns:
        TunerFnResult: Contains the tuner and fit arguments.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(fn_args.train_files[0], tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files[0], tf_transform_output)

    tuner = kt.Hyperband(
        hypermodel=get_tuner_model,
        objective=kt.Objective('val_loss', direction='min'),
        max_epochs=10,
        factor=3,
        directory=fn_args.working_dir,
        project_name='stroke_disaster_kt'
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            'callbacks': [early_stop]
        }
    )

Overwriting tuner.py


In [28]:
# Initialize the Tuner component
# This component performs hyperparameter tuning using the specified tuning logic and dataset
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),  # The path to the module file containing the tuning logic
    examples=transform.outputs['transformed_examples'],  # The transformed dataset output from the Transform component
    transform_graph=transform.outputs['transform_graph'],  # The transformation graph output from the Transform component
    schema=schema_gen.outputs['schema'],  # The schema output from the SchemaGen component
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=100),  # Training arguments specifying the training split and number of steps
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)  # Evaluation arguments specifying the evaluation split and number of steps
)

# Run the Tuner component interactively
# This will perform hyperparameter tuning according to the logic defined in the module file and make the results available in the interactive context
interactive_context.run(tuner)

Trial 30 Complete [00h 00m 05s]
val_loss: 0.17012520134449005

Best val_loss So Far: 0.14301133155822754
Total elapsed time: 00h 01m 21s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\stroke-pipeline\.temp\6\stroke_disaster_kt
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
dense_units: 208
dropout_rate: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.14301133155822754
Trial summary
Hyperparameters:
num_layers: 2
dense_units: 208
dropout_rate: 0.6
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.14308755099773407
Trial summary
Hyperparameters:
num_layers: 1
dense_units: 80
dropout_rate: 0.1
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.1438918262720108
Trial summary
Hyperparameters:
num_layers: 2
dense_units: 176
dropout_rate: 0.30000000000000004
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.14504994451999664
Trial summary
Hyperparameters:
num_layers: 2
dense_units: 240
dropout_

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Model Training

In [29]:
# Initialize for creating trainer module
TRAINER_MODULE_FILE = "trainer.py"

In [30]:
%%writefile {TRAINER_MODULE_FILE}

import os
import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from keras.utils.vis_utils import plot_model
from transform import (
    CATEGORICAL_FEATURES,
    LABEL_KEY,
    NUMERICAL_FEATURES,
    transformed_name,
)
from tuner import input_fn

def get_serve_tf_examples_fn(model, tf_transform_output):
    """Create a function to parse and transform serialized tf.Examples for serving.

    Args:
        model (tf.keras.Model): The trained Keras model.
        tf_transform_output (tft.TFTransformOutput): TFTransform output object.

    Returns:
        function: A function to process serialized tf.Examples.
    """
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Transform and predict using serialized tf.Examples.

        Args:
            serialized_tf_examples (tf.Tensor): Serialized tf.Example tensors.

        Returns:
            dict: A dictionary with model predictions.
        """
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)  # Remove label key from feature specification
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        outputs = model(transformed_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn

def get_model(hyperparameters, show_summary=True):
    """Define and compile a Keras model based on hyperparameters.

    Args:
        hyperparameters (dict): Hyperparameter values.
        show_summary (bool, optional): Whether to print the model summary. Defaults to True.

    Returns:
        tf.keras.Model: The compiled Keras model.
    """
    input_features = []

    # Create input layers for categorical features
    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    # Create input layers for numerical features
    for feature in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(feature))
        )

    # Define the model architecture
    concatenate = layers.concatenate(input_features)
    deep = layers.Dense(hyperparameters['dense_units'], activation='relu')(concatenate)

    for _ in range(hyperparameters['num_layers']):
        deep = layers.Dense(hyperparameters['dense_units'], activation='relu')(deep)

    deep = layers.Dropout(hyperparameters['dropout_rate'])(deep)
    outputs = layers.Dense(1, activation='sigmoid')(deep)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparameters['learning_rate']),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    if show_summary:
        model.summary()

    return model

def run_fn(fn_args):
    """Train and save the model based on provided arguments.

    Args:
        fn_args (NamedTuple): Contains arguments for training and saving the model.
    """
    hyperparameters = fn_args.hyperparameters['values']
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, 64)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, 64)

    model = get_model(hyperparameters)

    # Define callbacks for TensorBoard, early stopping, and model checkpointing
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        update_freq='batch'
    )

    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        mode='max',
        verbose=1,
        patience=10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor='val_binary_accuracy',
        mode='max',
        verbose=1,
        save_best_only=True
    )

    callbacks = [tensorboard_callback, early_stop_callback, model_checkpoint_callback]

    # Train the model
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
        epochs=hyperparameters['tuner/initial_epoch'],
        verbose=1
    )

    # Define the serving signature
    signatures = {
        'serving_default': get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
        ),
    }

    # Save the trained model
    model.save(
        fn_args.serving_model_dir,
        save_format='tf',
        signatures=signatures
    )

    # Plot and save the model architecture
    plot_model(
        model,
        to_file='images/model_plot.png',
        show_shapes=True,
        show_layer_names=True
    )

Overwriting trainer.py


In [31]:
# Initialize the Trainer component
# This component trains a model using the specified training logic, transformed data, and hyperparameters
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),  # The path to the module file containing the training logic
    examples=transform.outputs['transformed_examples'],  # The transformed dataset output from the Transform component
    transform_graph=transform.outputs['transform_graph'],  # The transformation graph output from the Transform component
    schema=schema_gen.outputs['schema'],  # The schema output from the SchemaGen component
    hyperparameters=tuner.outputs['best_hyperparameters'],  # The best hyperparameters identified by the Tuner component
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=100),  # Training arguments specifying the training split and number of steps
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)  # Evaluation arguments specifying the evaluation split and number of steps
)

# Run the Trainer component interactively
# This will train the model according to the specified logic, data, and hyperparameters, and make the results available in the interactive context
interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 gender_xf (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 ever_married_xf (InputLayer)   [(None, 3)]          0           []                               
                                                                                                  
 work_type_xf (InputLayer)      [(None, 6)]          0           []                               
                                                                                                  
 Residence_type_xf (InputLayer)  [(None, 3)]         0           []                               
                                                                                            

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\stroke-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\stroke-pipeline\Trainer\model\7\Format-Serving\assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Analysis and Validation Model

In [32]:
# Initialize the Resolver component
# This component resolves the latest blessed model based on the provided strategy and channels
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,  # Strategy for selecting the latest blessed model
    model=Channel(type=Model),  # Channel providing model artifacts
    model_blessing=Channel(type=ModelBlessing)  # Channel providing model blessing artifacts
).with_id("Latest_blessed_model_resolver")  # Assign an ID to the resolver component

# Run the Resolver component interactively
# This will select the latest blessed model according to the specified strategy and make it available in the interactive context
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [33]:
import tensorflow_model_analysis as tfma  # Import TensorFlow Model Analysis for model evaluation

# Define slicing specifications for evaluation
# This specifies how the data should be sliced for evaluation, allowing metrics to be computed for different slices of the data
slicing_specs = [
    tfma.SlicingSpec(),  # Default slicing (overall metrics)
    tfma.SlicingSpec(feature_keys=['gender', 'ever_married'])  # Metrics sliced by 'gender' and 'ever_married' features
]

# Define metrics specifications for evaluation
# This specifies which metrics to compute and their configurations during evaluation
metrics_specs = [
    tfma.MetricsSpec(metrics=[
        tfma.MetricConfig(class_name='AUC'),  # Area Under the Curve
        tfma.MetricConfig(class_name='ExampleCount'),  # Count of examples
        tfma.MetricConfig(class_name='TruePositives'),  # Count of true positives
        tfma.MetricConfig(class_name='FalsePositives'),  # Count of false positives
        tfma.MetricConfig(class_name='TrueNegatives'),  # Count of true negatives
        tfma.MetricConfig(class_name='FalseNegatives'),  # Count of false negatives
        tfma.MetricConfig(class_name='BinaryAccuracy',
            threshold=tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5}  # Threshold for the metric
                ),
                change_threshold=tfma.GenericChangeThreshold(
                    direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                    absolute={'value': 0.0001}  # Change threshold for the metric
                )
            )
        )
    ])
]

# Define the evaluation configuration
# This specifies the model specifications, slicing specifications, and metrics specifications to be used for model evaluation
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='stroke')],  # Model specification with the label key 'stroke'
    slicing_specs=slicing_specs,  # Define how data should be sliced for evaluation
    metrics_specs=metrics_specs  # Define which metrics to compute and their configurations
)

## Evaluation Model

In [34]:
# Initialize the Evaluator component
# This component evaluates the performance of a trained model against a baseline model using the specified evaluation configuration
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],  # The raw dataset output from the CsvExampleGen component
    model=trainer.outputs["model"],  # The trained model output from the Trainer component
    baseline_model=model_resolver.outputs["model"],  # The baseline (latest blessed) model output from the Resolver component
    eval_config=eval_config,  # The evaluation configuration specifying metrics and slicing specs
)

# Run the Evaluator component interactively
# This will evaluate the trained model against the baseline model according to the provided configuration and make the results available in the interactive context
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [35]:
import tensorflow_model_analysis as tfma  # Import TensorFlow Model Analysis

# Retrieve the URI of the evaluation results from the Evaluator component
eval_result = evaluator.outputs['evaluation'].get()[0].uri

# Load the evaluation results using TensorFlow Model Analysis
tfma_result = tfma.load_eval_result(eval_result)

# Render the slicing metrics from the evaluation results
# This visualizes the performance metrics sliced by the specified features
tfma.view.render_slicing_metrics(tfma_result)

# Render the fairness indicators using TensorFlow Model Analysis add-ons
# This visualizes fairness metrics to assess if the model's predictions are fair across different slices
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Yes_X_Male', 'slice': 'ever_married_X_gender:Yes_X_Mal…

## Export Model

In [36]:
from tfx.components import Pusher  # Import the Pusher component for model deployment
from tfx.proto import pusher_pb2  # Import protobuf definitions for Pusher component configuration

# Initialize the Pusher component
# This component is responsible for pushing the trained model to the specified destination for serving
pusher = Pusher(
    model=trainer.outputs['model'],  # The trained model output from the Trainer component
    model_blessing=evaluator.outputs['blessing'],  # The model blessing output from the Evaluator component, indicating that the model is ready for deployment
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='serving_model_dir/stroke-detection-model'  # The directory where the model will be pushed for serving
        )
    )
)

# Run the Pusher component interactively
# This will push the trained model to the specified filesystem destination, making it available for serving
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))